In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
import pandas as pd

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url)
soup=BeautifulSoup(r.text,'html.parser')

In [5]:
neighborhood_list=[[]]
counter=0
for link in soup.find_all('td'):
  if link.contents:
    content=link.contents[0]
    if content.name is None or content.name=='Not assigned':
      neighborhood_list[-1].append(content.strip())
      counter+=1
    elif content.name is not None:
      neighborhood_list[-1].append(link.a.attrs['title'].strip())
      counter+=1
    if counter%3==0:
      neighborhood_list.append([])
  else:
    break
del neighborhood_list[-1]
print(neighborhood_list)

[['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront (Toronto)'], ['M5A', 'Downtown Toronto', 'Regent Park'], ['M6A', 'North York', 'Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor'], ['M7A', "Queen's Park (Toronto)", 'Not assigned'], ['M8A', 'Not assigned', 'Not assigned'], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough, Toronto', 'Rouge, Toronto'], ['M1B', 'Scarborough, Toronto', 'Malvern, Toronto'], ['M2B', 'Not assigned', 'Not assigned'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens'], ['M4B', 'East York', 'Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson'], ['M5B', 'Downtown Toronto', 'Garden District'], ['M6B', 'North York', 'Glencairn'], ['M7B', 'Not assigned', 'Not assigned'], ['M8B', 'Not assigned', 'Not assigned'], ['M9B', 'Etobicoke', 'Cloverdale'], ['M9B', 'Et

In [6]:
neighbor_df=pd.DataFrame(neighborhood_list,columns=['PostalCode', 'Borough', 'Neighborhood'])
neighbor_df.shape

(288, 3)

In [7]:
na_index=neighbor_df[neighbor_df['Borough']=='Not assigned'].index
neighbor_df.drop(na_index,inplace=True)
neighbor_df.shape

(211, 3)

In [8]:
neighbor_df.loc[neighbor_df['Neighborhood']=='Not assigned','Neighborhood']=neighbor_df.loc[neighbor_df['Neighborhood']=='Not assigned','Borough']
neighbor_df['Neighborhood']=neighbor_df['Neighborhood'].apply(lambda x: x.split(',')[0] if ',' in x else x)
neighbor_df.head(10)
neighbor_group_df=neighbor_df.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x: ','.join(x))
neighbor_group_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,"Scarborough, Toronto","Rouge,Malvern"
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto),Rouge Hill,Port Union"
2,M1E,"Scarborough, Toronto","Guildwood,Morningside,West Hill"
3,M1G,"Scarborough, Toronto",Woburn
4,M1H,"Scarborough, Toronto",Cedarbrae


# To get the coordinates for each postal code

In [9]:
!pip install geocoder

    100% |████████████████████████████████| 102kB 12.1MB/s a 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [10]:
import geocoder 

def get_coord(postal_code):
  lat_lng_coords = None
  while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

  return lat_lng_coords[0],lat_lng_coords[1]

In [12]:

coord_df=pd.read_csv('https://cocl.us/Geospatial_data')
neighbor_df_merged=neighbor_group_df.merge(coord_df,how='left',left_on='PostalCode',right_on='Postal Code')
neighbor_df_merged.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,"Scarborough, Toronto","Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto),Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,"Scarborough, Toronto","Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,"Scarborough, Toronto",Woburn,M1G,43.770992,-79.216917
4,M1H,"Scarborough, Toronto",Cedarbrae,M1H,43.773136,-79.239476


In [13]:
CLIENT_ID = '0T10U13NSE5SK52EQCRH0AT25CZ0B0XD1S3VMZ0LF5G4UIDJ' 
CLIENT_SECRET = 'BPSGUQYVXBR0ZAMORCKKVZ031SRDAAPWX5IIOEBC1IATSRGW'
VERSION = '20180605' # Foursquare API version
radius=500
limit=100

In [14]:
def getNearbyVenues(postcodes, boroughs, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postcode, borough, name, lat, lng in zip(postcodes, boroughs, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postcode,
            borough,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode',
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(postcodes=neighbor_df_merged['PostalCode'],
                                   boroughs=neighbor_df_merged['Borough'],
                                   names=neighbor_df_merged['Neighborhood'],
                                   latitudes=neighbor_df_merged['Latitude'],
                                   longitudes=neighbor_df_merged['Longitude']
                                  )

Rouge,Malvern
Highland Creek (Toronto),Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter – Sullivan
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The B

In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#add back PostalCode
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()

In [18]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store
1,M1C,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant
2,M1E,Intersection,Spa,Rental Car Location,Electronics Store,Breakfast Spot
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run
4,M1H,Bakery,Hakka Restaurant,Caribbean Restaurant,Athletics & Sports,Bank


## KMeans to cluster the post codes

In [20]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#neighborhoods_venues_sorted.drop(columns=['Cluster Labels'],inplace=True)


toronto_merged = neighbor_df_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), how='right',on='PostalCode')

## Visualization

In [23]:
!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

    100% |████████████████████████████████| 92kB 11.7MB/s ta 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
